# Copying figures from Holst et al "Impacts of summer water limitation on the carbon balance of a Scots pine forest in the southern upper Rhine plain"

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2020)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath   = os.path.join('../')
print('datapath is set to %s'%datapath)

# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install plotly 
# !pip install cufflinks
#!pip install colorspacious
#!pip install seaborn

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile

from Ags_model import runAgs, calc_LE

In [ ]:
from FilterData import Filter_wrap

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_EC           = Read_LoobosEddFinal    (years,datapath)
    df_Stor         = Read_LooStor           (years,datapath)
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    df_NEE          = Read_Loobos_halfhourly (years,datapath)
    df_meteo        = Read_Loobos_meteo      (years,datapath)
    df_soil         = Read_Loobos_soil       (years,datapath) 
    df_profile      = Read_Loobos_profile    (years,datapath)
    progress.append('dataloaded')

In [ ]:
#from FilterData import Filter_wrap
CO2,Locorr,VPD,Ustar,df_profile_filter,df_meteo_filter,df_Comb_filter,df_EC_filter=Filter_wrap(df_Comb,df_profile,df_meteo,df_EC,filterversion='default')
#NOTE: df_Stor is NOT FILTERED

# Start of figure generation

## Figure 2

In [ ]:
#set aspect ratio and dpi
aspectratio=21/9
fig_wid=6 #we want 6 inch width
print(fig_wid/aspectratio) # around 2.6
fig_hei=2.6
dpi=144 #can be as high as 300, standard is 72 which is way too low

In [ ]:
# plotting F_NEE(micromol /m2 /s) over Ta (deg C), Ts (deg C), VPD (hPa), separated into two classes: PAR>1000 and PAR500-1000
#reminder to self: maybe make a LUE curve to justify the classes >1000 and 500-1000? Or unnecessary, quote literature?

#F_NEE data sourced from: df_Comb: NEE
#VPD data sourced from: df_Comb: VPD
#Ta (air temp) data sourced from: df_Comb: Tair
#Ts (soil temp) data sourced from: df_Comb: Tsoil
#PAR data sourced from: df_meteo: PAR

df_fig2 = pd.concat([df_Comb_filter['NEE'],df_Comb_filter['GPP_f'],df_Comb_filter['VPD'],df_Comb_filter['Tair'],df_Comb_filter['Tsoil'], df_meteo_filter['PAR']],axis=1,sort=False)

In [ ]:
#df_fig2.plot()

In [ ]:
#df_test = df_fig2.loc[df_fig2['PAR']<1000]
#df_test2 = df_test.loc[df_test['PAR']>500]

#df_test2e = df_fig2.loc[(500<df_fig2['PAR']<1000)]

#df_test2e.equals(df_test2)
#this throws error

### start of figure 2e and f

In [ ]:
#fig 2 e)

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2e = df_fig2.loc[df_fig2['PAR']<1000]

#make aliases
Tair=df_fig2e['Tair']
NEE=df_fig2e['NEE']
VPD=df_fig2e['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2e_cat1=df_fig2e.loc[(Tair>=10)&(Tair<=15)]
fig2e_cat2=df_fig2e.loc[(Tair>15)&(Tair<=20)]
fig2e_cat3=df_fig2e.loc[(Tair>20)&(Tair<=25)]
fig2e_cat4=df_fig2e.loc[Tair>25]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [10, 15 ,20, 25 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2e_cat1['VPD'],fig2e_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2e_cat2['VPD'],fig2e_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2e_cat3['VPD'],fig2e_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2e_cat4['VPD'],fig2e_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_ylim(-45,30)
ax.set_xlim(-1.5,45)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'VPD [Pa or hPA?]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2e: NEE vs VPD (color by Ta), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[25],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('Tair')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

In [ ]:
#fig 2 f)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2f = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair=df_fig2f['Tair']
NEE=df_fig2f['NEE']
VPD=df_fig2f['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2f_cat1=df_fig2f.loc[(Tair>=10)&(Tair<=15)]
fig2f_cat2=df_fig2f.loc[(Tair>15)&(Tair<=20)]
fig2f_cat3=df_fig2f.loc[(Tair>20)&(Tair<=25)]
fig2f_cat4=df_fig2f.loc[Tair>25]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [10, 15 ,20, 25 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2f_cat1['VPD'],fig2f_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2f_cat2['VPD'],fig2f_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2f_cat3['VPD'],fig2f_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2f_cat4['VPD'],fig2f_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_ylim(-45,30)
ax.set_xlim(-1.5,45)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'VPD [Pa or hPA?]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2f: NEE vs VPD (color by Ta), 500<PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[25],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('Tair')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

In [ ]:
cmap = mpl.colormaps['coolwarm'].resampled(4)
#for i,c in enumerate(cmap.get_colors):
cmap(-1050)

In [ ]:
cmap

In [ ]:
colorz = [(0.2298057, 0.298717966, 0.753683153), (1, 1, 1), (0.705673158, 0.01555616, 0.150232812)]
cmapz = mpl.colors.LinearSegmentedColormap.from_list('test', colorz, N=4)
cmapz

In [ ]:
cmap = mpl.colors.ListedColormap(['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326'], name='coolwarm_4', N=None)
cmap(0)

In [ ]:
#adapted figure: plot GPP instead of NEE (thus eliminating effects respiration, as NEE = GPP - respiration (ecosys.)

#GPP data sourced from: df_Comb: GPP_f

### end of figure 2e and f

### start of figure 2a and b

In [ ]:
#fig 2 a)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2a = df_fig2.loc[df_fig2['PAR']<1000]

#make aliases
Tair=df_fig2a['Tair']
NEE=df_fig2a['NEE']
VPD=df_fig2a['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2a_cat1=df_fig2a.loc[(VPD>=0)&(VPD<=5)]
fig2a_cat2=df_fig2a.loc[(VPD>5)&(VPD<=10)]
fig2a_cat3=df_fig2a.loc[(VPD>10)&(VPD<=15)]
fig2a_cat4=df_fig2a.loc[VPD>15]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2a_cat1['Tair'],fig2a_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2a_cat2['Tair'],fig2a_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2a_cat3['Tair'],fig2a_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2a_cat4['Tair'],fig2a_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_ylim(-30,20)
ax.set_xlim(-20,40)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'Tair [$\degree$C]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2a: NEE vs Tair (color by VPD), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('VPD')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

In [ ]:
#fig 2 b)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2b = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair=df_fig2b['Tair']
NEE=df_fig2b['NEE']
VPD=df_fig2b['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2b_cat1=df_fig2b.loc[(VPD>=0)&(VPD<=5)]
fig2b_cat2=df_fig2b.loc[(VPD>5)&(VPD<=10)]
fig2b_cat3=df_fig2b.loc[(VPD>10)&(VPD<=15)]
fig2b_cat4=df_fig2b.loc[VPD>15]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2b_cat1['Tair'],fig2b_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2b_cat2['Tair'],fig2b_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2b_cat3['Tair'],fig2b_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2b_cat4['Tair'],fig2b_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_ylim(-30,20)
ax.set_xlim(-20,40)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'Tair [$\degree$C]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2b: NEE vs Tair (color by VPD), 500<PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('VPD')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

### end of figure 2a and b

### start of figure 2c and d

In [ ]:
#fig 2 c)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2c = df_fig2.loc[df_fig2['PAR']<1000]
#df_fig2c = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair=df_fig2c['Tair']
Tsoil=df_fig2c['Tsoil']
NEE=df_fig2c['NEE']
VPD=df_fig2c['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2c_cat1=df_fig2c.loc[(VPD>=0)&(VPD<=5)]
fig2c_cat2=df_fig2c.loc[(VPD>5)&(VPD<=10)]
fig2c_cat3=df_fig2c.loc[(VPD>10)&(VPD<=15)]
fig2c_cat4=df_fig2c.loc[VPD>15]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2c_cat1['Tsoil'],fig2c_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2c_cat2['Tsoil'],fig2c_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2c_cat3['Tsoil'],fig2c_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2c_cat4['Tsoil'],fig2c_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_xlim(0,20)
ax.set_ylim(-35,25)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'Tsoil [$\degree$C]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2c: NEE vs Tsoil (color by VPD), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('VPD')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

In [ ]:
#fig 2 d)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2d = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair=df_fig2d['Tair']
Tsoil=df_fig2d['Tsoil']
NEE=df_fig2d['NEE']
VPD=df_fig2d['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2d_cat1=df_fig2d.loc[(VPD>=0)&(VPD<=5)]
fig2d_cat2=df_fig2d.loc[(VPD>5)&(VPD<=10)]
fig2d_cat3=df_fig2d.loc[(VPD>10)&(VPD<=15)]
fig2d_cat4=df_fig2d.loc[VPD>15]

# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05]) #coords are in proportion to fig size (fraction of fig dimension). 0,0 being bottom left corner, 1,1 being top right. format is [x1,y1,x2,y2] where x1,y1 is origin point of axes, and x2,y2 is the width and height of axes, respectively.

p1=ax.scatter(fig2d_cat1['Tsoil'],fig2d_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y)
p2=ax.scatter(fig2d_cat2['Tsoil'],fig2d_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3) #scatter(x,y)
p3=ax.scatter(fig2d_cat3['Tsoil'],fig2d_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3) #scatter(x,y)
p4=ax.scatter(fig2d_cat4['Tsoil'],fig2d_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_xlim(0,20)
ax.set_ylim(-35,25)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'Tsoil [$\degree$C]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2d: NEE vs Tsoil (color by VPD), 500<PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap_cbar,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb.set_label('VPD')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(300)
plt.plot()

### end of figure 2c and d

## plot figure 2 a-f all in one (for NEE and GPP)

In [ ]:
#fig 2 a-f)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2_PAR10 = df_fig2.loc[df_fig2['PAR']<1000]
df_fig2_PAR5_10 = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair_10=df_fig2_PAR10['Tair']
Tsoil_10=df_fig2_PAR10['Tsoil']
NEE_10=df_fig2_PAR10['NEE']
VPD_10=df_fig2_PAR10['VPD']

Tair5_10=df_fig2_PAR5_10['Tair']
Tsoil5_10=df_fig2_PAR5_10['Tsoil']
NEE5_10=df_fig2_PAR5_10['NEE']
VPD5_10=df_fig2_PAR5_10['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2ac_cat1=df_fig2_PAR10.loc[(VPD_10>=0)&(VPD_10<=5)]
fig2ac_cat2=df_fig2_PAR10.loc[(VPD_10>5)&(VPD_10<=10)]
fig2ac_cat3=df_fig2_PAR10.loc[(VPD_10>10)&(VPD_10<=15)]
fig2ac_cat4=df_fig2_PAR10.loc[VPD_10>15]

fig2bd_cat1=df_fig2_PAR5_10.loc[(VPD5_10>=0)&(VPD5_10<=5)]
fig2bd_cat2=df_fig2_PAR5_10.loc[(VPD5_10>5)&(VPD5_10<=10)]
fig2bd_cat3=df_fig2_PAR5_10.loc[(VPD5_10>10)&(VPD5_10<=15)]
fig2bd_cat4=df_fig2_PAR5_10.loc[VPD5_10>15]

fig2e_cat1=df_fig2_PAR10.loc[(Tair_10>=10)&(Tair_10<=15)]
fig2e_cat2=df_fig2_PAR10.loc[(Tair_10>15)&(Tair_10<=20)]
fig2e_cat3=df_fig2_PAR10.loc[(Tair_10>20)&(Tair_10<=25)]
fig2e_cat4=df_fig2_PAR10.loc[Tair_10>25]

fig2f_cat1=df_fig2_PAR5_10.loc[(Tair5_10>=10)&(Tair5_10<=15)]
fig2f_cat2=df_fig2_PAR5_10.loc[(Tair5_10>15)&(Tair5_10<=20)]
fig2f_cat3=df_fig2_PAR5_10.loc[(Tair5_10>20)&(Tair5_10<=25)]
fig2f_cat4=df_fig2_PAR5_10.loc[Tair5_10>25]


# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds_abcd = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
cbar_bounds_ef = [10, 15 ,20, 25 ]
norm_abcd = mpl.colors.BoundaryNorm(cbar_bounds_abcd, cmap.N)
norm_ef = mpl.colors.BoundaryNorm(cbar_bounds_ef, cmap.N)

#create figure
fig = plt.figure(layout='constrained') #optional arg: facecolor='grey'
fig.set_dpi(300)
fig.suptitle('Sens. of NEE to Tair, Tsoil, VPD, classified by PAR')
fig.set_figwidth(12)
fig.set_figheight(7.8)

#figTnest[0].suptitle('500<PAR<1000')
#figTnest[1].suptitle('PAR<1000')
#figTnestL,figtnestR = figTnest[0].subfigures(nrows=1, ncols=2)
axs=fig.subplot_mosaic([['A panel', 'B panel'],
                            ['A panel', 'B panel'],
                            ['C panel', 'D panel'],
                            ['C panel', 'D panel'],
                            ['cbar1',   'cbar1'  ],
                            ['E panel', 'F panel'],
                            ['E panel', 'F panel'],
                            ['cbar2',   'cbar2'  ],])


# Draw figure
# a) top left
p1a=axs['A panel'].scatter(fig2ac_cat1['Tair'],fig2ac_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y) . fig2ac_cat1 contains values for both a and c
p2a=axs['A panel'].scatter(fig2ac_cat2['Tair'],fig2ac_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3a=axs['A panel'].scatter(fig2ac_cat3['Tair'],fig2ac_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4a=axs['A panel'].scatter(fig2ac_cat4['Tair'],fig2ac_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['A panel'].set_xlabel('Tair(C)')
axs['A panel'].set_ylabel('NEE(umolm-2s-1)')

# b) top right
p1b=axs['B panel'].scatter(fig2bd_cat1['Tair'],fig2bd_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2b=axs['B panel'].scatter(fig2bd_cat2['Tair'],fig2bd_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3b=axs['B panel'].scatter(fig2bd_cat3['Tair'],fig2bd_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4b=axs['B panel'].scatter(fig2bd_cat4['Tair'],fig2bd_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['B panel'].set_xlabel('Tair(C)')
axs['B panel'].set_ylabel('NEE(umolm-2s-1)')

# c) mid left
p1c=axs['C panel'].scatter(fig2ac_cat1['Tsoil'],fig2ac_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2c=axs['C panel'].scatter(fig2ac_cat2['Tsoil'],fig2ac_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3c=axs['C panel'].scatter(fig2ac_cat3['Tsoil'],fig2ac_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4c=axs['C panel'].scatter(fig2ac_cat4['Tsoil'],fig2ac_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['C panel'].set_xlabel('Tsoil(C)')
axs['C panel'].set_ylabel('NEE(umolm-2s-1)')

# d) mid right
p1d=axs['D panel'].scatter(fig2bd_cat1['Tsoil'],fig2bd_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2d=axs['D panel'].scatter(fig2bd_cat2['Tsoil'],fig2bd_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3d=axs['D panel'].scatter(fig2bd_cat3['Tsoil'],fig2bd_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4d=axs['D panel'].scatter(fig2bd_cat4['Tsoil'],fig2bd_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['D panel'].set_xlabel('Tsoil(C)')
axs['D panel'].set_ylabel('NEE(umolm-2s-1)')

# e) bottom leftVPD
p1e=axs['E panel'].scatter(fig2e_cat1['VPD'],fig2e_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2e=axs['E panel'].scatter(fig2e_cat2['VPD'],fig2e_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3e=axs['E panel'].scatter(fig2e_cat3['VPD'],fig2e_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4e=axs['E panel'].scatter(fig2e_cat4['VPD'],fig2e_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['E panel'].set_xlabel('VPD(hPa)')
axs['E panel'].set_ylabel('NEE(umolm-2s-1)')

# f) bottom right
p1f=axs['F panel'].scatter(fig2f_cat1['VPD'],fig2f_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2f=axs['F panel'].scatter(fig2f_cat2['VPD'],fig2f_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3f=axs['F panel'].scatter(fig2f_cat3['VPD'],fig2f_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4f=axs['F panel'].scatter(fig2f_cat4['VPD'],fig2f_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['F panel'].set_xlabel('VPD(hPa)')
axs['F panel'].set_ylabel('NEE(umolm-2s-1)')


axs['A panel'].set_title('500<PAR<1000')
axs['B panel'].set_title('PAR<1000')


#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

# Axes limits, titles, layout
axs['A panel'].set_ylim(-30,25) #a)
axs['A panel'].set_xlim(-15,40)
axs['B panel'].set_ylim(-30,25) #b)
axs['B panel'].set_xlim(-15,40)
axs['C panel'].set_ylim(-30,25) #c)
axs['C panel'].set_xlim(0,20)
axs['D panel'].set_ylim(-30,25) #d)
axs['D panel'].set_xlim(0,20)
axs['E panel'].set_ylim(-40,25) #e)
axs['E panel'].set_xlim(0,40)
axs['F panel'].set_ylim(-40,25) #f)
axs['F panel'].set_xlim(0,40)

#ax1.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
#ax1.set_xlabel(r'Tair [$\degree$C]')
#ax1.set_title('Fig2a: NEE vs Tair (color by VPD), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb_abcd = mpl.colorbar.ColorbarBase(axs['cbar1'], cmap=cmap_cbar,
                                norm=norm_abcd,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds_abcd+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds_abcd,  # optional
                                spacing='proportional',
                                orientation='horizontal')
axs['cbar1'].set_position([0,0.43,1,0.02])
axs['cbar1'].set_xlabel('VPD')


cb_ef = mpl.colorbar.ColorbarBase(axs['cbar2'], cmap=cmap_cbar,
                                norm=norm_ef,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds_ef+[25],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds_ef,  # optional
                                spacing='proportional',
                                orientation='horizontal')
axs['cbar2'].set_position([0,0.07,1,0.02])
axs['cbar2'].set_xlabel('Tair')

#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')

plt.plot()

In [ ]:
#fig 2 a-f)
#AGAIN BUT FOR GPP

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2_PAR10 = df_fig2.loc[df_fig2['PAR']<1000]
df_fig2_PAR5_10 = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

#make aliases
Tair_10=df_fig2_PAR10['Tair']
Tsoil_10=df_fig2_PAR10['Tsoil']
NEE_10=df_fig2_PAR10['GPP_f'] #refactor this variable later to GPP_10
VPD_10=df_fig2_PAR10['VPD']

Tair5_10=df_fig2_PAR5_10['Tair']
Tsoil5_10=df_fig2_PAR5_10['Tsoil']
NEE5_10=df_fig2_PAR5_10['GPP_f'] #refactor this variable later to GPP5_10
VPD5_10=df_fig2_PAR5_10['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2ac_cat1=df_fig2_PAR10.loc[(VPD_10>=0)&(VPD_10<=5)]
fig2ac_cat2=df_fig2_PAR10.loc[(VPD_10>5)&(VPD_10<=10)]
fig2ac_cat3=df_fig2_PAR10.loc[(VPD_10>10)&(VPD_10<=15)]
fig2ac_cat4=df_fig2_PAR10.loc[VPD_10>15]

fig2bd_cat1=df_fig2_PAR5_10.loc[(VPD5_10>=0)&(VPD5_10<=5)]
fig2bd_cat2=df_fig2_PAR5_10.loc[(VPD5_10>5)&(VPD5_10<=10)]
fig2bd_cat3=df_fig2_PAR5_10.loc[(VPD5_10>10)&(VPD5_10<=15)]
fig2bd_cat4=df_fig2_PAR5_10.loc[VPD5_10>15]

fig2e_cat1=df_fig2_PAR10.loc[(Tair_10>=10)&(Tair_10<=15)]
fig2e_cat2=df_fig2_PAR10.loc[(Tair_10>15)&(Tair_10<=20)]
fig2e_cat3=df_fig2_PAR10.loc[(Tair_10>20)&(Tair_10<=25)]
fig2e_cat4=df_fig2_PAR10.loc[Tair_10>25]

fig2f_cat1=df_fig2_PAR5_10.loc[(Tair5_10>=10)&(Tair5_10<=15)]
fig2f_cat2=df_fig2_PAR5_10.loc[(Tair5_10>15)&(Tair5_10<=20)]
fig2f_cat3=df_fig2_PAR5_10.loc[(Tair5_10>20)&(Tair5_10<=25)]
fig2f_cat4=df_fig2_PAR5_10.loc[Tair5_10>25]


# visual reference for segments and boundaries:  |-(10-15)--|---(15-20)--|--(20-25)--|-(25+)---
# meaning 4 segments with 4 boundaries (upper segment unbounded: 25+ but lower segment bounded: 10 or greater

#colorbar settings
#for some reason the object created and assigned to cmap (matplotlib.colors.LinearSegmentedColormap) does not have an attribute .colors (which it should have? or maybe it's just meant to be a constructor?
#as such, to properly assign the 'outside range' cbar colors I'd have to either do a workaround, or I can just manually asign my colors. I'm opting for the latter.
colors=['#3A4CC0', '#AAC6FD', '#F6B79C', '#B30326']
cmap= mpl.colors.ListedColormap(colors, name='coolwarm_4', N=None)
colors_cbar=['#3A4CC0', '#AAC6FD', '#F6B79C', '#F6B79C']
cmap_cbar= mpl.colors.ListedColormap(colors_cbar, name='coolwarm_4', N=None) #have to do this weird trick to get the single-side unbounded cbar to work
cmap_cbar.set_over('#B30326')

#cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1)),mpl.colors.to_hex(cmap(2)),mpl.colors.to_hex(cmap(3))] #may be unnecessary, use cmap(0) cmap(1) cmap(2) etc

cbar_bounds_abcd = [0, 5 ,10, 15 ] #boundaries excluding the abs min and abs max
cbar_bounds_ef = [10, 15 ,20, 25 ]
norm_abcd = mpl.colors.BoundaryNorm(cbar_bounds_abcd, cmap.N)
norm_ef = mpl.colors.BoundaryNorm(cbar_bounds_ef, cmap.N)

#create figure
fig = plt.figure(layout='constrained') #optional arg: facecolor='grey'
fig.set_dpi(300)
fig.suptitle('Sens. of GPP to Tair, Tsoil, VPD, classified by PAR')
fig.set_figwidth(12)
fig.set_figheight(7.8)

#figTnest[0].suptitle('500<PAR<1000')
#figTnest[1].suptitle('PAR<1000')
#figTnestL,figtnestR = figTnest[0].subfigures(nrows=1, ncols=2)
axs=fig.subplot_mosaic([['A panel', 'B panel'],
                            ['A panel', 'B panel'],
                            ['C panel', 'D panel'],
                            ['C panel', 'D panel'],
                            ['cbar1',   'cbar1'  ],
                            ['E panel', 'F panel'],
                            ['E panel', 'F panel'],
                            ['cbar2',   'cbar2'  ],])


# Draw figure
# a) top left
p1a=axs['A panel'].scatter(fig2ac_cat1['Tair'],fig2ac_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3) #scatter(x,y) . fig2ac_cat1 contains values for both a and c
p2a=axs['A panel'].scatter(fig2ac_cat2['Tair'],fig2ac_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3a=axs['A panel'].scatter(fig2ac_cat3['Tair'],fig2ac_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4a=axs['A panel'].scatter(fig2ac_cat4['Tair'],fig2ac_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['A panel'].set_xlabel('Tair(C)')
axs['A panel'].set_ylabel('GPP(umolm-2s-1)')

# b) top right
p1b=axs['B panel'].scatter(fig2bd_cat1['Tair'],fig2bd_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2b=axs['B panel'].scatter(fig2bd_cat2['Tair'],fig2bd_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3b=axs['B panel'].scatter(fig2bd_cat3['Tair'],fig2bd_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4b=axs['B panel'].scatter(fig2bd_cat4['Tair'],fig2bd_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['B panel'].set_xlabel('Tair(C)')
axs['B panel'].set_ylabel('GPP(umolm-2s-1)')

# c) mid left
p1c=axs['C panel'].scatter(fig2ac_cat1['Tsoil'],fig2ac_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2c=axs['C panel'].scatter(fig2ac_cat2['Tsoil'],fig2ac_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3c=axs['C panel'].scatter(fig2ac_cat3['Tsoil'],fig2ac_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4c=axs['C panel'].scatter(fig2ac_cat4['Tsoil'],fig2ac_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['C panel'].set_xlabel('Tsoil(C)')
axs['C panel'].set_ylabel('GPP(umolm-2s-1)')

# d) mid right
p1d=axs['D panel'].scatter(fig2bd_cat1['Tsoil'],fig2bd_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2d=axs['D panel'].scatter(fig2bd_cat2['Tsoil'],fig2bd_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3d=axs['D panel'].scatter(fig2bd_cat3['Tsoil'],fig2bd_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4d=axs['D panel'].scatter(fig2bd_cat4['Tsoil'],fig2bd_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['D panel'].set_xlabel('Tsoil(C)')
axs['D panel'].set_ylabel('GPP(umolm-2s-1)')

# e) bottom leftVPD
p1e=axs['E panel'].scatter(fig2e_cat1['VPD'],fig2e_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2e=axs['E panel'].scatter(fig2e_cat2['VPD'],fig2e_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3e=axs['E panel'].scatter(fig2e_cat3['VPD'],fig2e_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4e=axs['E panel'].scatter(fig2e_cat4['VPD'],fig2e_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['E panel'].set_xlabel('VPD(hPa)')
axs['E panel'].set_ylabel('GPP(umolm-2s-1)')

# f) bottom right
p1f=axs['F panel'].scatter(fig2f_cat1['VPD'],fig2f_cat1['NEE'],marker='.',s=5,c=colors[0],alpha=0.3)
p2f=axs['F panel'].scatter(fig2f_cat2['VPD'],fig2f_cat2['NEE'],marker='s',s=5,c=colors[1],alpha=0.3)
p3f=axs['F panel'].scatter(fig2f_cat3['VPD'],fig2f_cat3['NEE'],marker='d',s=5,c=colors[2],alpha=0.3)
p4f=axs['F panel'].scatter(fig2f_cat4['VPD'],fig2f_cat4['NEE'],marker='+',s=10,c=colors[3],alpha=0.3)
axs['F panel'].set_xlabel('VPD(hPa)')
axs['F panel'].set_ylabel('GPP(umolm-2s-1)')


axs['A panel'].set_title('500<PAR<1000')
axs['B panel'].set_title('PAR<1000')


#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

# Axes limits, titles, layout
axs['A panel'].set_ylim(-30,25) #a)
axs['A panel'].set_xlim(-15,40)
axs['B panel'].set_ylim(-30,25) #b)
axs['B panel'].set_xlim(-15,40)
axs['C panel'].set_ylim(-30,25) #c)
axs['C panel'].set_xlim(0,20)
axs['D panel'].set_ylim(-30,25) #d)
axs['D panel'].set_xlim(0,20)
axs['E panel'].set_ylim(-40,25) #e)
axs['E panel'].set_xlim(0,40)
axs['F panel'].set_ylim(-40,25) #f)
axs['F panel'].set_xlim(0,40)

#ax1.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
#ax1.set_xlabel(r'Tair [$\degree$C]')
#ax1.set_title('Fig2a: NEE vs Tair (color by VPD), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb_abcd = mpl.colorbar.ColorbarBase(axs['cbar1'], cmap=cmap_cbar,
                                norm=norm_abcd,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds_abcd+[15],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds_abcd,  # optional
                                spacing='proportional',
                                orientation='horizontal')
axs['cbar1'].set_position([0,0.43,1,0.02])
axs['cbar1'].set_xlabel('VPD')


cb_ef = mpl.colorbar.ColorbarBase(axs['cbar2'], cmap=cmap_cbar,
                                norm=norm_ef,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=cbar_bounds_ef+[25],
                                extend='max', #use 'both' for left and right
                                ticks=cbar_bounds_ef,  # optional
                                spacing='proportional',
                                orientation='horizontal')
axs['cbar2'].set_position([0,0.07,1,0.02])
axs['cbar2'].set_xlabel('Tair')

#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')

plt.plot()

### end of figure 2 all in one

## figure 3

In [ ]:
#figure 3 has two components, F_NEE over soil moist, and gs (canopy conducance in mmol m-2s-1) over soil moist.
#the graphical type is bar and whisker plots.
# I'll plot the F_NEE part for now, and do the gs part at another time

df_soil_SM = pd.concat([df_soil['SM-Lit'], df_soil['SM-003'], df_soil['SM-020'], df_soil['SM-050'], df_soil['SM-100']],axis=1,sort=False)
df_soil_SM.dropna(axis='index', how='all',inplace=True)
df_fig3 = pd.concat([df_Comb_filter['NEE'],df_Comb_filter['GPP_f'],df_meteo_filter['PAR'],df_soil_SM],axis=1,sort=False)
df_fig3 = df_fig3.loc['2005-04-11 14:30:00':] #before this date there's no SM data
df_fig3

In [ ]:
#this doesn't work, a NaN value == np.NaN gives false for some reason, use dropna
#SM_lit = df_soil.loc[df_soil['SM-Lit'] != np.NaN]
#SM_003 = df_soil.loc[df_soil['SM-003'] != np.NaN]
#SM_020 = df_soil.loc[df_soil['SM-020'] != np.NaN]
#SM_050 = df_soil.loc[df_soil['SM-050'] != np.NaN]
#SM_100 = df_soil.loc[df_soil['SM-100'] != np.NaN]

#df_fig2_PAR5_10 = df_fig2.loc[(df_fig2['PAR']<1000) & (df_fig2['PAR']>500)]

In [ ]:
#df_soil['SM-Lit']
#df_soil['SM-003']
#df_soil['SM-020']
#df_soil['SM-050']
#df_soil['SM-100']

### explore SM data visually

In [ ]:
SM_lit = df_soil.loc[:,['SM-Lit']].dropna()
#SM_lit2 = df_soil_filtered.loc[:,['SM-Lit']].dropna()
SM_003 = df_soil.loc[:,['SM-003']].dropna()
SM_020 = df_soil.loc[:,['SM-020']].dropna()
SM_050 = df_soil.loc[:,['SM-050']].dropna()
SM_100 = df_soil.loc[:,['SM-100']].dropna()

In [ ]:
SM_lit.plot()

In [ ]:
SM_003.plot()

In [ ]:
SM_020.plot()

In [ ]:
SM_050.plot()

In [ ]:
SM_100.plot()

### end of exploring SM data visually

### plot fig 3

In [ ]:
#instead of F_NEE and Gs lets plot F_NEE and GPP

bins = pd.IntervalIndex.from_tuples([(0.02, 0.03), (0.03, 0.04), (0.04, 0.05), (0.05, 0.06), (0.06, 0.07), (0.07, 0.08), (0.08, 0.09), (0.09, 0.1),(0.1,0.3)])

df_fig3_1 = df_fig3.copy()
df_fig3_1.loc[:,'SM-020_bintag']=pd.cut(df_fig3_1.loc[:,'SM-020'],bins=bins)
condlist = [df_fig3_1.loc[:,'PAR']<=500, df_fig3_1.loc[:,'PAR']>500]
choicelist = ['<=500', '>500']
df_fig3_1.loc[:,'PAR_tag']=np.select(condlist,choicelist,np.nan)
df_fig3_1.dropna(axis='index',how='any',subset=['PAR','PAR_tag'],inplace=True)

sns.set_theme(style="ticks", palette="pastel")

fig, axes = plt.subplots(nrows=2,ncols=1,figsize=(12,10)) 

p1=sns.boxplot(data=df_fig3_1, x='SM-020_bintag', y='NEE', hue='PAR_tag', palette=['red','blue'], ax=axes[0], gap=0.5, fliersize=0.01 )
p2=sns.boxplot(data=df_fig3_1, x='SM-020_bintag', y='GPP_f', hue='PAR_tag', palette=['red','blue'], ax=axes[1], gap=0.5, fliersize=0.01 )

axes[0].set_ylim(-40,30)
axes[1].set_ylim(-15,40)


#print the number count above the boxplot (top plot)
df_fig3N_under5=df_fig3_1.loc[df_fig3_1['PAR_tag']=='<=500']
df_fig3N_under5.dropna(axis='index',how='any',subset=['PAR','PAR_tag','NEE'],inplace=True)
nobs_un = df_fig3N_under5.loc[:,'SM-020_bintag'].value_counts().values
nobs_un = [str(x) for x in nobs_un.tolist()]
pos_un = range(len(nobs_un))
for tick,label in zip(pos_un,axes[0].get_xticklabels()):
    axes[0].text(tick,29,nobs_un[tick],color='black', horizontalalignment="right", verticalalignment = "top",rotation="vertical", bbox=dict(facecolor='pink'))
df_fig3N_over5=df_fig3_1.loc[df_fig3_1['PAR_tag']=='>500']
df_fig3N_over5.dropna(axis='index',how='any',subset=['PAR','PAR_tag','NEE'],inplace=True)
nobs_ov = df_fig3N_over5.loc[:,'SM-020_bintag'].value_counts().values
nobs_ov = [str(x) for x in nobs_ov.tolist()]
pos_ov = range(len(nobs_ov))
for tick,label in zip(pos_ov,axes[0].get_xticklabels()):
    axes[0].text(tick+0.5,28,nobs_ov[tick],color='black', horizontalalignment="right", verticalalignment = "top",rotation="vertical", bbox=dict(facecolor='cyan'))

df_fig3G_under5=df_fig3_1.loc[df_fig3_1['PAR_tag']=='<=500']
df_fig3G_under5.dropna(axis='index',how='any',subset=['PAR','PAR_tag','GPP_f'],inplace=True)
nobs_un = df_fig3G_under5.loc[:,'SM-020_bintag'].value_counts().values
nobs_un = [str(x) for x in nobs_un.tolist()]
pos_un = range(len(nobs_un))
for tick,label in zip(pos_un,axes[1].get_xticklabels()):
    axes[1].text(tick,39,nobs_un[tick],color='black', horizontalalignment="right", verticalalignment = "top",rotation="vertical", bbox=dict(facecolor='pink'))
df_fig3G_over5=df_fig3_1.loc[df_fig3_1['PAR_tag']=='>500']
df_fig3G_over5.dropna(axis='index',how='any',subset=['PAR','PAR_tag','GPP_f'],inplace=True)
nobs_ov = df_fig3G_over5.loc[:,'SM-020_bintag'].value_counts().values
nobs_ov = [str(x) for x in nobs_ov.tolist()]
pos_ov = range(len(nobs_ov))
for tick,label in zip(pos_ov,axes[1].get_xticklabels()):
    axes[1].text(tick+0.5,38,nobs_ov[tick],color='black', horizontalalignment="right", verticalalignment = "top",rotation="vertical", bbox=dict(facecolor='cyan'))


axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation = 30)
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation = 30)
#sns.despine(offset={'left':10}, trim=False, top=False, right=False, ax=p1)
axes[0].legend(loc='lower left')
axes[1].legend(loc='lower left')
axes[0].set_xlabel(r'$\Theta$ (soil moist depth=20cm)')
axes[1].set_xlabel(r'$\Theta$ (soil moist depth=20cm)')
plt.tight_layout()

plt.show()


In [ ]:
#instead of F_NEE and Gs lets plot F_NEE and GPP

bins = pd.IntervalIndex.from_tuples([(0.02, 0.03), (0.03, 0.04), (0.04, 0.05), (0.05, 0.06), (0.06, 0.07), (0.07, 0.08), (0.08, 0.09), (0.09, 0.1),(0.1,0.3)])

df_fig3_1 = df_fig3.copy()
df_fig3_1.loc[:,'SM-020_bintag']=pd.cut(df_fig3_1.loc[:,'SM-020'],bins=bins)
condlist = [df_fig3_1.loc[:,'PAR']<=500, df_fig3_1.loc[:,'PAR']>500]
choicelist = ['<=500', '>500']
df_fig3_1.loc[:,'PAR_tag']=np.select(condlist,choicelist,np.nan)
df_fig3_1.dropna(axis='index',how='any',subset=['PAR','PAR_tag'],inplace=True)

sns.set_theme(style="ticks", palette="pastel")

fig, axes = plt.subplots(nrows=2,ncols=1,figsize=(12,10)) 

p1=sns.boxplot(data=df_fig3_1, x='SM-020_bintag', y='NEE', hue='PAR_tag', palette=['red','blue'], ax=axes[0], gap=0.5, fliersize=0.01 )
p2=sns.boxplot(data=df_fig3_1, x='SM-020_bintag', y='GPP_f', hue='PAR_tag', palette=['red','blue'], ax=axes[1], gap=0.5, fliersize=0.01 )

axes[0].set_ylim(-40,30)
axes[1].set_ylim(-15,40)


#print the number count above the boxplot (top plot)
df_fig3N_under5=df_fig3_1.loc[df_fig3_1['PAR_tag']=='<=500']
df_fig3N_under5.dropna(axis='index',how='any',subset=['PAR','PAR_tag','NEE'],inplace=True)
nobs_un = df_fig3N_under5.loc[:,'SM-020_bintag'].value_counts().values
nobs_un = [str(x) for x in nobs_un.tolist()]
pos_un = range(len(nobs_un))
for tick,label in zip(pos_un,axes[0].get_xticklabels()):
    axes[0].text(tick,29,nobs_un[tick],color='black', horizontalalignment="right", verticalalignment = "top",rotation="vertical", bbox=dict(facecolor='pink'))
df_fig3N_over5=df_fig3_1.loc[df_fig3_1['PAR_tag']=='>500']
df_fig3N_over5.dropna(axis='index',how='any',subset=['PAR','PAR_tag','NEE'],inplace=True)
nobs_ov = df_fig3N_over5.loc[:,'SM-020_bintag'].value_counts().values
nobs_ov = [str(x) for x in nobs_ov.tolist()]
pos_ov = range(len(nobs_ov))
for tick,label in zip(pos_ov,axes[0].get_xticklabels()):
    axes[0].text(tick+0.5,28,nobs_ov[tick],color='black', horizontalalignment="right", verticalalignment = "top",rotation="vertical", bbox=dict(facecolor='cyan'))

df_fig3G_under5=df_fig3_1.loc[df_fig3_1['PAR_tag']=='<=500']
df_fig3G_under5.dropna(axis='index',how='any',subset=['PAR','PAR_tag','GPP_f'],inplace=True)
nobs_un = df_fig3G_under5.loc[:,'SM-020_bintag'].value_counts().values
nobs_un = [str(x) for x in nobs_un.tolist()]
pos_un = range(len(nobs_un))
for tick,label in zip(pos_un,axes[1].get_xticklabels()):
    axes[1].text(tick,39,nobs_un[tick],color='black', horizontalalignment="right", verticalalignment = "top",rotation="vertical", bbox=dict(facecolor='pink'))
df_fig3G_over5=df_fig3_1.loc[df_fig3_1['PAR_tag']=='>500']
df_fig3G_over5.dropna(axis='index',how='any',subset=['PAR','PAR_tag','GPP_f'],inplace=True)
nobs_ov = df_fig3G_over5.loc[:,'SM-020_bintag'].value_counts().values
nobs_ov = [str(x) for x in nobs_ov.tolist()]
pos_ov = range(len(nobs_ov))
for tick,label in zip(pos_ov,axes[1].get_xticklabels()):
    axes[1].text(tick+0.5,38,nobs_ov[tick],color='black', horizontalalignment="right", verticalalignment = "top",rotation="vertical", bbox=dict(facecolor='cyan'))


axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation = 30)
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation = 30)
#sns.despine(offset={'left':10}, trim=False, top=False, right=False, ax=p1)
axes[0].legend(loc='lower left')
axes[1].legend(loc='lower left')
axes[0].set_xlabel(r'$\Theta$ (soil moist depth=20cm)')
axes[1].set_xlabel(r'$\Theta$ (soil moist depth=20cm)')
plt.tight_layout()

plt.show()


### end of plot fig 3

### Start fig 7

In [ ]:
#figure 3 has two components, F_NEE over soil moist, and gs (canopy conducance in mmol m-2s-1) over soil moist.
#the graphical type is bar and whisker plots.
# I'll plot the F_NEE part for now, and do the gs part at another time

df_fig7 = pd.concat([df_meteo_filter['P(mast)'],df_Comb_filter['Tair']],axis=1,sort=False)
#df_fig7

P=df_fig7[['P(mast)']]
Tair=df_fig7[['Tair']]
P_bymonth=P.resample('M').sum()
P_mean=P_bymonth['P(mast)'].mean()
P_bymonth['dP_%']=(P_bymonth['P(mast)']/P_mean)*100
#because of how the plot function operates, it will only plot negatives on negative values so we must adapt it:
P_bymonth['dP_r%']=P_bymonth['dP_%']-100 # r% meaning relative %, 0=100%, 100=200% -20=80% etc. more negative than -100 means below 0% of the dataset and loses physical meaning
Tair_bymonth=Tair.resample('M').mean()
Tair_mean=Tair['Tair'].mean()
Tair_bymonth['deltaT']=Tair_bymonth['Tair']-Tair_mean

#select years and compile into one chart
st_date='2001-01-01'
ed_date='2020-12-31'

fig7_data = pd.concat([P_bymonth.loc[st_date:ed_date,'dP_r%'],Tair_bymonth.loc[st_date:ed_date,'deltaT']],axis=1,sort=False)


### plots to see data ranges

In [ ]:
df_meteo['P(mast)'].plot()

In [ ]:
df_meteo['P(guter)'].plot()

In [ ]:
#'P(mast)00', 'P(guter)00',
#       'P(mast)05', 'P(guter)05', 'P(mast)10', 'P(guter)10', 'P(mast)15',
#       'P(guter)15', 'P(mast)20', 'P(guter)20', 'P(mast)25', 'P(guter)25',
#       'P(mast)', 'P(guter)',

In [ ]:
fig7_data.loc[:,'dP_r%'].plot()

In [ ]:
fig7_data.loc[:,'deltaT'].plot()

### end of plots to see data ranges

### plot fig 7

In [ ]:
# Figure 7

#prep data
year_L=2017
year_R=2018
#fig7_data=fig7_data.loc[fig7_data.index.year==year_L]

#create fig and axes
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(12,6)) 
axes[0].set(xlim=(-15,15), ylim=(-100,100)) #add 100 to both ylim values in your mind
axes[0].set_title(str(year_L))
axes[0].set_xlabel(r'$\Delta T_a$ ($\degree$C)')
axes[0].set_ylabel(r'$\delta$P (%)')
axes[1].set(xlim=(-15,15), ylim=(-100,100))
axes[1].set_title(str(year_R))
axes[1].set_xlabel(r'$\Delta T_a$ ($\degree$C)')
axes[1].set_ylabel(r'$\delta$P (%)')

axes[0].axhline(0, linestyle='-', color='k') # horizontal lines
axes[0].axvline(0, linestyle='-', color='k') # vertical lines
axes[1].axhline(0, linestyle='-', color='k') # horizontal lines
axes[1].axvline(0, linestyle='-', color='k') # vertical lines


#x=fig7_data.loc[:,'deltaT']
#y=fig7_data.loc[:,'dP_r%']


#we want vector arrows with a dot at the end. this is not supported (arrows are polygons, not lines with markers), so we make shafts:

def plot_anomgraph(ax,fig7_data):
    x= fig7_data.loc[:,'deltaT']
    y= fig7_data.loc[:,'dP_r%']
    # mpl.quiver([X, Y], U, V, [C], **kwargs)  where X, Y are arrays of x&y origin points respec. U is x end coord, V is y end coord
    #X and Y are x and y coords of the origin point. In my case, it' always 0,0 (NB:not 0,100!), because y axis is % not factorial
    #but X and Y need to be same length as amount of vectors.
    #so we multiply [0] by n (or len(data))
    ax.quiver([0]*len(fig7_data),[0]*len(fig7_data),
                   x, #x component aka U
                   y, #y-component aka V
                   scale=1,
                   scale_units='xy',
                   angles = 'xy',
                   headlength = 0.0,
                   headaxislength = 0.0)
    
    #plot markers at the end:
    ax.scatter(x, y,c='k')
    
    #plot annotations
    annotations = x.index.strftime('%b') #x index's month in 3 letter format
    for xi, yi, text in zip(x, y, annotations):
        xj=np.where(xi>0, 20, -20) #extend position by either 20 or -20 depending on what quadrant of figure it is (pos or neg x)
        yj=np.where(yi>0, 20, -20)
        ax.annotate(text,
                    xy=(xi, yi), xycoords='data', ha='center', va='center',
                    xytext=(xj, yj), textcoords='offset pixels')
        
    #since we used a trick before to set the dP_r% values such that 100% is 0 in dP_r%, we paint over the y labels to correct it.
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 30)
    
    #axes[0].annotate("O", (0, 100),xytext=(x,y), ha='center', va='bottom' ,arrowprops=dict(arrowstyle= '-', color='k',lw=2))
    
    str_vals=[textobj.get_text() for textobj in ax.get_yticklabels()]
    #tick_pos=[textobj.get_position() for textobj in ax.get_yticklabels()] #might need this if switching to set_yticks()
    int_vals=[int(s.replace('\u2212', '-'))+100 for s in str_vals] #for some reason output of previous line can be '-100' but it's a different dash used in unicode instead of a ascii dash. find characters with str.encode('unicode-escape') and replace.
    tick_strs=[str(i) for i in int_vals]
    ax.set_yticklabels(tick_strs) #there's another recommended method axis.set_yticks(positions, labels) but it doesn't like my input

data_L=fig7_data.loc[fig7_data.index.year==year_L].dropna(axis='index') #on all rows where deltaT is Nan, deltaP is -100, suspicious.
data_R=fig7_data.loc[fig7_data.index.year==year_R].dropna(axis='index')
plot_anomgraph(axes[0],data_L)
plot_anomgraph(axes[1],data_R)

plt.plot()

In [ ]:
# Figure 7

#prep data
year_L=2013
year_R=2015
#fig7_data=fig7_data.loc[fig7_data.index.year==year_L]

#create fig and axes
fig, axes = plt.subplots(nrows=1,ncols=2,figsize=(12,6)) 
axes[0].set(xlim=(-15,15), ylim=(-100,100)) #add 100 to both ylim values in your mind
axes[0].set_title(str(year_L))
axes[0].set_xlabel(r'$\Delta T_a$ ($\degree$C)')
axes[0].set_ylabel(r'$\delta$P (%)')
axes[1].set(xlim=(-15,15), ylim=(-100,100))
axes[1].set_title(str(year_R))
axes[1].set_xlabel(r'$\Delta T_a$ ($\degree$C)')
axes[1].set_ylabel(r'$\delta$P (%)')

axes[0].axhline(0, linestyle='-', color='k') # horizontal lines
axes[0].axvline(0, linestyle='-', color='k') # vertical lines
axes[1].axhline(0, linestyle='-', color='k') # horizontal lines
axes[1].axvline(0, linestyle='-', color='k') # vertical lines


#x=fig7_data.loc[:,'deltaT']
#y=fig7_data.loc[:,'dP_r%']


#we want vector arrows with a dot at the end. this is not supported (arrows are polygons, not lines with markers), so we make shafts:

def plot_anomgraph(ax,fig7_data):
    x= fig7_data.loc[:,'deltaT']
    y= fig7_data.loc[:,'dP_r%']
    # mpl.quiver([X, Y], U, V, [C], **kwargs)  where X, Y are arrays of x&y origin points respec. U is x end coord, V is y end coord
    #X and Y are x and y coords of the origin point. In my case, it' always 0,0 (NB:not 0,100!), because y axis is % not factorial
    #but X and Y need to be same length as amount of vectors.
    #so we multiply [0] by n (or len(data))
    ax.quiver([0]*len(fig7_data),[0]*len(fig7_data),
                   x, #x component aka U
                   y, #y-component aka V
                   scale=1,
                   scale_units='xy',
                   angles = 'xy',
                   headlength = 0.0,
                   headaxislength = 0.0)
    
    #plot markers at the end:
    ax.scatter(x, y,c='k')
    
    #plot annotations
    annotations = x.index.strftime('%b') #x index's month in 3 letter format
    for xi, yi, text in zip(x, y, annotations):
        xj=np.where(xi>0, 20, -20) #extend position by either 20 or -20 depending on what quadrant of figure it is (pos or neg x)
        yj=np.where(yi>0, 20, -20)
        ax.annotate(text,
                    xy=(xi, yi), xycoords='data', ha='center', va='center',
                    xytext=(xj, yj), textcoords='offset pixels')
        
    #since we used a trick before to set the dP_r% values such that 100% is 0 in dP_r%, we paint over the y labels to correct it.
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 30)
    
    #axes[0].annotate("O", (0, 100),xytext=(x,y), ha='center', va='bottom' ,arrowprops=dict(arrowstyle= '-', color='k',lw=2))
    
    str_vals=[textobj.get_text() for textobj in ax.get_yticklabels()]
    #tick_pos=[textobj.get_position() for textobj in ax.get_yticklabels()] #might need this if switching to set_yticks()
    int_vals=[int(s.replace('\u2212', '-'))+100 for s in str_vals] #for some reason output of previous line can be '-100' but it's a different dash used in unicode instead of a ascii dash. find characters with str.encode('unicode-escape') and replace.
    tick_strs=[str(i) for i in int_vals]
    ax.set_yticklabels(tick_strs) #there's another recommended method axis.set_yticks(positions, labels) but it doesn't like my input

data_L=fig7_data.loc[fig7_data.index.year==year_L].dropna(axis='index') #on all rows where deltaT is Nan, deltaP is -100, suspicious.
data_R=fig7_data.loc[fig7_data.index.year==year_R].dropna(axis='index')
plot_anomgraph(axes[0],data_L)
plot_anomgraph(axes[1],data_R)

plt.plot()

### end fig 7